## Simulation of one round of blockchain-based federated learning 

In [1]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [2]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(
        f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available and set the default tensor type to CUDA
print('Using device: %s' % device)
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("Cuda set as default device")
else:
    torch.set_default_device('cpu')
    print("Cuda not available, CPU set as default device")

Cuda available: True
NVIDIA GeForce MX150
major and minor cuda capability of the device: (6, 1)
Using device: cuda
Cuda set as default device


### Config variables for models and training


In [ ]:
from os.path import join as pjoin
import json
from client.train import train
from client.model.perceptron import MultiLayerPerceptron
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json, weights_zero_init,  weights_init, update_lr
from client.aggregators import federated_aggregate
from client.dataloader import get_cifar10_dataloaders, get_cifar10_datasets

input_size = 32 * 32 * 3
hidden_size = [50]
num_classes = 10
num_epochs = 1
learning_rate = 1e-3
learning_rate_decay = 0.95
reg = 0.001
modelpath = 'client/models/'
train_flag = True


### Load data


In [ ]:

root = 'client/data/'
num_training = 49000
num_validation = 1000
batch_size = 200
train_dataset, val_dataset, test_dataset = get_cifar10_datasets(
    root, num_training, num_validation)
train_loader, val_loader, test_loader = get_cifar10_dataloaders(
    root, batch_size, num_training, num_validation, device)

dataloaders = {
    'train': train_loader,
    'validation': val_loader,
    'test': test_loader
}


### Create two local models


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from client.model.perceptron import Perceptron

model1 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model1.to(device)
print(model1)

model2 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model2.to(device)
print(model2)



### Train first local model


In [ ]:
model1_name = 'bcfl_model3'

# Training
model1.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model1.parameters(), lr=learning_rate, weight_decay=reg)

train(model1, modelpath, model1_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)



### Train second local model

In [ ]:
model2_name = 'bcfl_model4'
# Training
model2.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model2.parameters(), lr=learning_rate, weight_decay=reg)

train(model2, modelpath, model2_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)

### Test the trained models

In [ ]:
from os.path import join as pjoin
modelpath = 'client/models/'

# Run the test code once you have your by setting train flag to false
# and loading the best model
model_ckpt = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model_ckpt = torch.load(pjoin(modelpath, model1_name + '.ckpt'))
model1.load_state_dict(model_ckpt)
model = model1
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))
    
    

model = model2
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))


### Forward the two local models to the blockchain


In [ ]:
# Send the saved model to Fabric-SDK via Gateway Client (REST call)

from client.services.gateway_client import submit_model, get_all_models, get_model


print(submit_model(model1_name, model1.state_dict()))

print(submit_model(model2_name, model2.state_dict()))



### Submit empty model to the blockchain

In [ ]:
# TODO this should be triggered automatically when the total users in the blockchain have triggered their updates

from client.utils import weights_zero_init

model3 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model3.to(device)
weights_zero_init(model3)
model3_name = 'bcfl_model_empty'
# print(model3)
print(submit_model(model3_name, model3.state_dict()))



### Aggregate the local models on the blockchain
TODO this should be triggered automatically when the total users in the blockchain have triggered their updates


In [ ]:
# TODO this should be triggered automatically when the total users in the blockchain have triggered their updates

from client.services.gateway_client import aggregate_models



aggregate_models([model1_name, model3_name])



### Download the new global model


In [ ]:

from os.path import join as pjoin
import json
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json

# retrieve future global model and convert it again to pytorch model

# TODO change
model_avg_name = model1_name + 'and' + model3_name

model_params = get_model(model_avg_name)

global_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
global_model.to(device)
print(global_model)

load_model_from_json(global_model, model_params)
print(global_model)
   


### Print the new global model and compare it

In [ ]:
from client.utils import count_parameters, compare_models

print(model1.state_dict()['layers.0.bias'])
print(global_model.state_dict()['layers.0.bias'])
# the global model should be half of the model1

print(count_parameters(model1))
print(count_parameters(global_model))


### Test the new global model

In [ ]:

model = global_model
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))

### Retrieve global model and convert it again to pytorch model

In [ ]:

from os.path import join as pjoin
import json
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json, count_parameters, compare_models


local_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
local_model = torch.load(pjoin(modelpath,  'ml_model2.ckpt'))
model.load_state_dict(local_model)

model_params = get_model('ml_model2')
# print(client_response)


bc_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
bc_model.to(device)
# print(bc_model)

load_model_from_json(bc_model, model_params)
print(bc_model)
print(count_parameters(model))
print(count_parameters(bc_model))
print(compare_models(model, bc_model))
# for parameter in model.parameters():
#     print(parameter)


## Transformers

### Create Bert Tiny model

In [3]:
from os.path import join as pjoin

from transformers import AutoModel # For BERTs
from transformers import AutoModelForSequenceClassification # For models fine-tuned on MNLI
from transformers import AutoTokenizer
import torch

# tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny") # v1 and v2
bert_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny") # v1 and v2
print(bert_model)
bert_modelname = 'bert_model'


/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

### Save the model in a local file, load it again and compare

In [4]:
# Send the saved model to Fabric-SDK via Gateway Client (REST call)

from client.services.gateway_client import submit_model, get_all_models, get_model

modelpath = 'client/models/'

# Save the model checkpoint
torch.save(bert_model.state_dict(), pjoin(modelpath, bert_modelname + '.pt'))
    
loaded_bert_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny") # v1 and v2

loaded_bert_model_params = torch.load(pjoin(modelpath, bert_modelname + '.pt'))
loaded_bert_model.load_state_dict(loaded_bert_model_params)

print(count_parameters(bert_model))
print(count_parameters(loaded_bert_model))
print(compare_models(bert_model, loaded_bert_model))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'count_parameters' is not defined

### Serialize and encode model, decode and try if its equal

In [15]:

import sys
from client.services.gateway_client import submit_model
from client.utils import serialize_model_numpy, deserialize_model_numpy, print_layer_size, serialize_model_msgpack, deserialize_model_msgpack, compare_models, compare_state_dicts, compare_weights


encoded = serialize_model_msgpack(bert_model.state_dict(), 0, 0)
# encoded = serialize_model_numpy(bert_model.state_dict())

print(sys.getsizeof(encoded) / 1024 / 1024)
print(type(encoded))
decoded_state_dict = deserialize_model_msgpack(encoded)
# decoded_state_dict = deserialize_model_numpy(encoded)


bert2_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny") # v1 and v2

bert2_model.load_state_dict(decoded_state_dict)

print(compare_state_dicts(bert_model.state_dict(), bert2_model.state_dict()))
print(compare_models(bert_model, bert2_model))
print(compare_weights(bert_model, bert2_model))

23.658535957336426
<class 'str'>


NotImplementedError: 

### Submit Bert Tiny model to the blockchain

In [33]:




import sys

from client.services.gateway_client import submit_model
from client.utils import print_layer_size, serialize_model_msgpack, deserialize_model_msgpack, compare_models, compare_state_dicts, compare_weights

# Send the saved model to Fabric-SDK via Gateway Client (REST call)
# print(submit_model(bert_modelname, bert_model.state_dict()))
print(submit_model(bert_modelname, bert_model.state_dict(), 0, 0))
# print_layer_size(bert_model.state_dict(), 0, 6, 'mb', 'base64')
# print(submit_compressed_model(bert_modelname, bert_model.state_dict()))


{'message': 'Model submitted succesfully'}


### Download the model and load it again


In [37]:
from os.path import join as pjoin
import json
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json, count_parameters, compare_models




blockchain_bert_model = get_model(bert_modelname)

decoded_state_dict = deserialize_model_msgpack(blockchain_bert_model['modelParams'])


bert2_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny") # v1 and v2

bert2_model.load_state_dict(decoded_state_dict)

print(compare_state_dicts(bert_model.state_dict(), bert2_model.state_dict()))
print(compare_models(bert_model, bert2_model))
print(compare_weights(bert_model, bert2_model))




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True
True
True
4386178


NameError: name 'bc_bert_model' is not defined

In [ ]:
import torch
# NVIDIA drivers not working
torch.set_default_device('cpu')
device = ('cpu')